## Introdução

Este notebook explora técnicas avançadas de manipulação de dados com Apache Spark no Databricks. Aprendemos a importar dados com schemas definidos e inferidos, realizar transformações, operações lógicas, renomeação de colunas, manipulações de datas e finalmente, salvar dados em diferentes formatos. Cada script é projetado para demonstrar uma funcionalidade específica do Spark, fornecendo ao usuário uma base sólida para análise de dados em larga escala.

#### Importando Dados e Definindo Schema

In [ ]:
from pyspark.sql.types import *
arqschema = "id INT, nome STRING, status STRING, cidade STRING, vendas INT, data STRING"
#o caminho pode mudar, download é a pasta que você baixou com dados de exemplo
despachantes = spark.read.csv("/FileStore/tables/despachantes.csv", header=False, schema=arqschema)
despachantes.show()

Aqui, definimos um schema explícito para os dados que estamos importando. Isso garante que cada coluna seja tratada com o tipo de dado correto. Utilizamos o método spark.read.csv() para carregar os dados de um arquivo CSV sem cabeçalho, especificando o schema definido.

#### Inferindo Schema Automaticamente

In [ ]:
#outro exemplo, inferindo schema, usando load e informado tipo
desp_autoschema = spark.read.load("/FileStore/tables/despachantes.csv",
                     format="csv", sep=",", inferSchema=True, header=False)
desp_autoschema.show()

In [ ]:
#comparando os schemas, outra forma
desp_autoschema.printSchema()
despachantes.printSchema()

Ao contrário do exemplo anterior, aqui permitimos que o Spark infira o schema dos dados automaticamente. Isso é útil para conjuntos de dados onde não conhecemos antecipadamente os tipos de dados das colunas. O método printSchema() é usado para visualizar o schema inferido.

#### Operações Lógicas com Colunas

In [ ]:
from pyspark.sql import functions as Func
#condição lógica com where
despachantes.select("id","nome","vendas").where(Func.col("vendas") > 20).show()
#& para and, | para or, e ~ para not
despachantes.select("id","nome","vendas").where((Func.col("vendas") > 20) & (Func.col("vendas") < 40)).show()

#### Renomeando e Manipulando Datas

In [ ]:
#renomear coluna
novodf = despachantes.withColumnRenamed("nome","nomes")
novodf.columns

In [ ]:
from pyspark.sql.functions import *
#coluna data está como string, vamos transformar em texto
despachantes2 = despachantes.withColumn("data2", to_timestamp(Func.col("data"),"yyyy-MM-dd"))
despachantes2.printSchema()

Aqui, renomeamos uma coluna e transformamos outra de string para data usando to_timestamp(), o que facilita operações subsequentes de data.

In [ ]:
#operações sobre datas
despachantes2.select(year("data")).show()
despachantes2.select(year("data")).distinct().show()
despachantes2.select("nome",year("data")).orderBy("nome").show()
despachantes2.select("data").groupBy(year("data")).count().show()
despachantes2.select(Func.sum("vendas")).show()

#### Salvando Dados em Diferentes Formatos

In [ ]:
#salvar, são diretórios
despachantes.write.format("parquet").save("dfimportparquet")
despachantes.write.format("csv").save("dfimportcsv")
despachantes.write.format("json").save("dfimportjson")
despachantes.write.format("orc").save("dfimportorc")

In [ ]:
%fs ls

In [ ]:
%fs ls dbfs:/dfimportjson/

In [ ]:
#ler dados
par = spark.read.format("parquet").load("dfimportparquet/*.parquet")
par.show()
par.printSchema()

In [ ]:
#formato tabular
despachantes.show(1)

In [ ]:
#formato de lista
despachantes.take(1) #/head/firt

In [ ]:
#retorna todos dados como uma lista
despachantes.collect()

In [ ]:
#conta
despachantes.count()

In [ ]:
#trasnformações
#padrão crescente
despachantes.orderBy("vendas").show()

In [ ]:
#decrescente
despachantes.orderBy(Func.col("vendas").desc()).show()

In [ ]:
#se quiser cidade dec e valor dec
despachantes.orderBy(Func.col("cidade").desc(),Func.col("vendas").desc()).show()

In [ ]:
#agrupar dados
#ver vendas por cidade
despachantes.groupBy("cidade").agg(sum("vendas")).show()

#### Operações Avançadas de Ordenação e Agregação

In [ ]:
#ordernar por vendas decrecente
from pyspark.sql import functions as F
despachantes.groupBy("cidade").agg(F.sum("vendas").alias("total_vendas")).orderBy(F.desc("total_vendas")).show()

In [ ]:
#filter
despachantes.filter(Func.col("nome") == "Deolinda Vilela").show()

Esses scripts demonstram ordenação e agregação avançadas. Usamos orderBy() para ordenar os dados e groupBy() seguido de agg() para realizar agregações, como somar vendas por cidade.